<img src="images/hdd.jpg" width="20%" align="right">

DataFrame Storage
================

Efficient storage can dramatically improve performance, particularly when operating repeatedly from disk.

Decompressing text and parsing CSV files is expensive.  One of the most effective strategies with medium data is to use a binary storage format like HDF5.  Often the performance gains from doing this is sufficient so that you can switch back to using Pandas again instead of using `dask.dataframe`.

In this section we'll learn how to efficiently arrange and store your datasets in on-disk binary formats.  We'll use the following:

1.  [Pandas `HDFStore`](http://pandas.pydata.org/pandas-docs/stable/io.html#io-hdf5) format on top of `HDF5`
2.  Categoricals for storing text data numerically
3.  [Castra](http://github.com/Blosc/castra), an experimental data store optimized for `tgdask.dataframe`

### Setup

Create data if we don't have any

In [ ]:
from prep import accounts_csvs
accounts_csvs(3, 1000000, 500)

### Read CSV

First we read our csv data as before

In [ ]:
import os
filename = os.path.join('data', 'accounts.*.csv')
filename

In [ ]:
import dask.dataframe as dd
df = dd.read_csv(filename)
df.head()

### Write to HDF5

Pandas contains a specialized HDF5 format, `HDFStore`.  The ``dd.DataFrame.to_hdf`` method works exactly like the ``pd.DataFrame.to_hdf`` method.

In [ ]:
target = os.path.join('data', 'accounts.h5')
target

In [ ]:
%time df.to_hdf(target, '/data')

In [ ]:
df2 = dd.read_hdf(target, '/data')
df2.head()

### Compare CSV to HDF5 speeds

We do a simple computation that requires reading a column of our dataset and compare performance between CSV files and our newly created HDF5 file.  Which do you expect to be faster?

In [ ]:
%time df.amount.sum().compute()

In [ ]:
%time df2.amount.sum().compute()

Sadly they are about the same.  

The culprit here is `names` column, which is of `object` dtype and thus hard to store efficiently.  There are two problems here:

1.  How do we store text data like `names` efficiently on disk?
2.  Why did we have to read the `names` column when all we wanted was `amount`

### 1.  Store text efficiently with categoricals

We can use Pandas categoricals to replace our object dtypes with a numerical representation.  This takes a bit more time up front, but results in better performance.

More on categoricals at the [pandas docs](http://pandas-docs.github.io/pandas-docs-travis/categorical.html) and [this blogpost](http://matthewrocklin.com/blog/work/2015/06/18/Categoricals/).

In [ ]:
# Categorize data, then store in HDFStore
%time df.categorize(columns=['names']).to_hdf(target, '/data2')

In [ ]:
# It looks the same
df2 = dd.read_hdf(target, '/data2')
df2.head()

In [ ]:
# But loads more quickly
%time df2.amount.sum().compute()

This is significantly faster.  This tells us that it's not only the file type that we use but also how we represent our variables that influences storage performance.

However this can still be better.  We had to read all of the columns (`names` and `amount`) in order to compute the sum of one (`amount`).  We'll improve further on this with `castra`, an on-disk column-store.  First though we learn about how to set an index in a dask.dataframe.

`set_index`
------------

As we're about to learn, the index is even more important in `dask.dataframe` than it was in `pandas`.  The index determines how we parallelize our computations and how efficiently we can index into parts of our dataset.  

In [ ]:
# By default `DataFrame.set_index` is *not lazily evaluated*
%time df3 = df.set_index('id')

In [ ]:
df3.head()

### But now we can perform lookups with `.loc`

In [ ]:
df3.loc[100].head()

### 2.  Store in columns and using an index with castra

Additionally, once we have a proper index we can use [`castra`](http://github.com/Blosc/castra), an on-disk column-store that is partitioned along the index.

You will likely have to install castra using `pip`.

    pip install castra
    
*Disclaimer: Castra is experimental and under heavy development churn.  You should not use it in production.*

*We discuss Castra here to demonstrate the value of thinking hard about storage, not as an endorsement of its use.*

In [ ]:
%%time
if os.path.exists('accounts.castra'):
    import shutil
    shutil.rmtree('accounts.castra')

c = df3.to_castra('accounts.castra', categories=['names'])
df4 = c.to_dask()

In [ ]:
%%time
df4.head()

In [ ]:
%%time
df4.loc[0:4].compute()

In [ ]:
%%time
df4.amount.sum().compute()

In [ ]:
%%time
df4.names.drop_duplicates().compute()

## Conclusion

Storage choices strongly impact performance.  We evolved from text-based CSV files to binary-based Castra and saw our query times drop from 1s to 80ms.

We also used `DataFrame.set_index` to organize our data along a special column.  A common recipe for success with `dask.dataframe` is as follows:

1.  Read in your data however it was delivered to you

        df = dd.read_csv('myfiles.*.csv')
    
2.  Set your index 

        df2 = df.set_index('column-name')
        
3.  Base computation on Castra file

        c = df2.to_castra('/path/to/new/file.castra', 
                          categories=['list', 'of', 'columns', 'to', 'categorize'])
        df3 = c.to_dask()
        
4.  Perform efficient queries

        df3.loc['2014': '2015'].groupby().events.count()